# Why Expose A Model As API ? Understanding this...

R is one of the most widely used programming language for data analysis and visualizations and so is used by a majority of the data scientists to build ML models for prediction and forecasting purposes. 

The end consumers of these models maybe software developers using a different technology stack or others with no domain expertise in data science and ML algorithms.

You could replicate the code into other programming languages used widely around the world by software developers but here lies the problem - Majority of the programming languages do not have libraries to perform ML.

So one good solution is to expose the model as API. The most common type of API is REST/RESTful API. 
REST APIs allow two applications each other over the Internet using HTTP as their protocol.
In simpler terms, they deliver the requests from client to the server (specified in the URL) and then deliver the responses (as requested in the method - GET, PUT, POST) back to the client.

By exposing the R Model as API, we can let other applications to use model for the purposes it is designed for without worrying abut the underlying environment/architecture. 




# How to expose R Model As API ? Let's get started...
Do we have a package in R to help with this ? The answer is yes.

Plumber is package that 